In [3]:
p = 6
q = 8
guess = f"{p}{q}".encode()
type(guess)

bytes

# Holdem Odds Calculator

In [152]:
import math
from collections import Counter
# Constants
# suits = {"s": 0, "c": 1, "h": 2, "d": 3}
suits = ("s", "c", "h", "d")
# valRank = "23456789TJQK"
handRanks = ('Royal Flush',
            'Straight Flush',
            'Four of a Kind',
            'Full House',
            'Flush',
            'Straight',
            'Three of a Kind',
            'Two Pair',
            'Pair',
            'High Card')
ranks = {"t": 10, "j": 11, "q": 12, "k": 13, "a": 14}
for num in range(2, 10):
    ranks[str(num)] = num

def withSameSuit(*cards, n=0):
    """
    At least n out of all the given cards have the same suit.
    n=0 means ALL with the same suit.
    Return that suit or False.
    """
    if (n == 0) or (n > len(cards)):
        n = len(cards)
    suits = [card[1] for card in cards]
    c = Counter(suits)
    suitCount = sorted([(count, suit) for (suit, count) in c.items()],
                       reverse=True)
    if suitCount[0][0] >= n:
        return suitCount[0][1]
    return False

def binom(n, k):
    return math.factorial(n) // math.factorial(k) // math.factorial(n - k)

In [214]:
class Holdem:
    def __init__(self, n, hand1, hand2, board1, board2, board3):
        self.numOfPlayer = n
        self.hand = [card.lower() for card in [hand1, hand2]]
        self.board = [card.lower() for card in [board1, board2, board3]]
        self.knownCards = self.hand + self.board
        self.reportCount()

    def revealBoard(self, nextCard):
        if len(self.board) > 4:
            raise ValueError(f"Too many cards on board! The board already has {len(self.board)} cards.")
        self.board.append(nextCard.lower())
        self.knownCards.append(nextCard.lower())
        self.reportCount()
        if len(self.board) == 5:
            self.bestHand()
            
    def bestHand(self):
        bestRank = "Royal Flush"
        cardsFromBoard = ""
        print(f"Best Result - {bestRank}")
        print(f"Your Hand: {self.hand}")
        print(f"Board: {self.board}")
        print(f"With from board: {cardsFromBoard}")
    
            
    def countRoyalFlush(self):
        """
        Count number of possible Royal Flush.
        A, K, Q, J, 10, all the same suit.
        """
        prob = 0
        royalFlushRanks = ['a', 'k', 'q', 'j', 't']
        commonSuit = withSameSuit(*self.knownCards, n=len(self.knownCards)-2)
        if commonSuit: 
            # At least 3 of known cards have the same suit
            valuesOfCommonSuit = [card[0] for card in self.knownCards
                                 if card[1] == commonSuit]
            for value in valuesOfCommonSuit:
                if value in royalFlushRanks:
                    royalFlushRanks.remove(value)
        if 7 - len(self.knownCards) >= len(royalFlushRanks):
            # unknown cards >= cards needed to be royalFlush
            prob = 1/(52 - len(self.knownCards))
            if len(royalFlushRanks) == 2:
                prob *= 1 / (52-len(self.knownCards)-1)
            elif len(royalFlushRanks) > 2:
                raise ValueError(f"Need {len(royalFlushRanks)} more cards to complete Royal Flush!")
            elif len(royalFlushRanks) == 0:
                prob = 1
                
        return f"{prob*100:.3f}%"
    
    def countStraightFlush(self):
        """
        Count number of possible Straight Flush.
        Excluding Royal Flush.
        Five cards in a sequence, all in the same suit.
        """
        prob = 0
        commonSuit = withSameSuit(*self.knownCards, n=len(self.knownCards)-2)
        if commonSuit: 
            # At least 3 of known cards have the same suit
            valuesOfCommonSuit = [card[0] for card in self.knownCards
                                 if card[1] == commonSuit]
            if "a" in valuesOfCommonSuit:
                # excluding royal flush
                valuesOfCommonSuit.remove("a")
            rawValuesOfCommonSuit = [ranks[x] for x in valuesOfCommonSuit]
            knownRange = max(rawValuesOfCommonSuit) - min(rawValuesOfCommonSuit)
            if len(valuesOfCommonSuit) == len(self.knownCards)-2:
                # have 3/5 or 4/6 or 5/7 cards
                if knownRange == 4:
                    # 5 cards are certain
                    if len(valuesOfCommonSuit) == 3:
                        prob = 1/(52 - len(self.knownCards)) / (52-len(self.knownCards)-1)
                    elif len(valuesOfCommonSuit) == 4:
                        prob = 1/(52 - len(self.knownCards))
                    elif len(valuesOfCommonSuit) == 5:
                        prob = 1
                elif knownRange == 3:
                    # 4 cards are certain
                    if len(valuesOfCommonSuit) == 3:
                        if (13 == max(rawValuesOfCommonSuit) or
                            2 == min(rawValuesOfCommonSuit)):
                            prob = 1/binom(52-len(self.knownCards), 2)
                        else:
                            prob = 2/binom(52-len(self.knownCards), 2)
                    elif len(valuesOfCommonSuit) == 4:
                        if (13 == max(rawValuesOfCommonSuit) or
                            2 == min(rawValuesOfCommonSuit)):
                            # cannot take 14(ace)
                            prob = (7-len(self.knownCards))/(52-len(self.knownCards))
                        else:
                            prob = (2*(52-len(self.knownCards)-1)-1)/binom(52-len(self.knownCards),2)
                elif knownRange == 2:
                    # 3 cards are certain
                    if len(valuesOfCommonSuit) == 3:
                        if (13 == max(rawValuesOfCommonSuit) or
                            2 == min(rawValuesOfCommonSuit)):
                            # can only wait for 9,10
                            prob = 1/binom(52-len(self.knownCards), 2)
                        elif (12 == max(rawValuesOfCommonSuit) or
                              3 == min(rawValuesOfCommonSuit)):
                            # 9,K or 8,9
                            prob = 2/binom(52-len(self.knownCards), 2)
                        else:
                            # e.g., (6,7,8): 4,5; 5,9; 9,10
                            prob = 3/binom(52-len(self.knownCards), 2)

            elif len(valuesOfCommonSuit) > len(self.knownCards)-2:
                # have 4,5/5 or 5,6/6 or 7/7 cards
                if len(valuesOfCommonSuit) == 4:
                    # have 4/5 cards in straight
                    if knownRange == 4:
                        # 5 cards are certain
                        prob = 2/(52-5)
                    elif knownRange == 3:
                        # 4 cards are certain
                        if (13 == max(rawValuesOfCommonSuit) or
                            2 == min(rawValuesOfCommonSuit)):
                            prob = 2/(52-5)
                        else:
                            prob = (2*(52-len(self.knownCards)-1)-1)/binom(52-len(self.knownCards),2)
                else:
                    prob = 1
        
        return f"{prob*100:.3f}%"
    
    def countFourOfAKind(self):
        """
        All four cards of the same rank.
        """
        prob = 0
        
        return f"{prob*100:.3f}%"
    
    def countFullHouse(self):
        """
        Three of a kind with a pair.
        """
        prob = 0
        
        return f"{prob*100:.3f}%"
    
    def countFlush(self):
        """
        Any five cards of the same suit, but not in a sequence
        """
        prob = 0
        
        return f"{prob*100:.3f}%"
    
    def countStraight(self):
        """
        Five cards in a sequence, but not of the same suit.        
        """
        prob = 0
        
        return f"{prob*100:.3f}%"
    
    def countThreeOfAKind(self):
        """
        Three cards of the same rank.
        """
        prob = 0
        
        return f"{prob*100:.3f}%"
    
    def countTwoPairs(self):
        """
        Two different pairs.
        """
        prob = 0
        
        return f"{prob*100:.3f}%"
    
    def countPair(self):
        """
        One pair.
        """
        prob = 0
        
        return f"{prob*100:.3f}%"
    
    # otherwise, compare the High Card
    def findHighCard(self):
        cards = [card[0] for card in self.knownCards]
        cardsValue = sorted([ranks[x] for x in cards], reverse=True)
        return cardsValue
    
    def reportCount(self):
        print(f"""
        Royal Flush:     {self.countRoyalFlush()}
        Straight Flush:  {self.countStraightFlush()}
        Four of a Kind:  {self.countFourOfAKind()}
        Full House:      {self.countFullHouse()}
        Flush:           {self.countFlush()}
        Straight:        {self.countStraight()}
        Three of a Kind: {self.countThreeOfAKind()}
        Two Pair:        {self.countTwoPairs()}
        Pair:            {self.countPair()}
        High Card:       {self.findHighCard()}
        """)

In [215]:
# test
firstTurn = Holdem(5, 'KS', 'QD', 'TS', 'QS', '4C')
firstTurn.__dict__


        Royal Flush:     0.046%
        Straight Flush:  0.093%
        Four of a Kind:  0.000%
        Full House:      0.000%
        Flush:           0.000%
        Straight:        0.000%
        Three of a Kind: 0.000%
        Two Pair:        0.000%
        Pair:            0.000%
        High Card:       [13, 12, 12, 10, 4]
        


{'numOfPlayer': 5,
 'hand': ['ks', 'qd'],
 'board': ['ts', 'qs', '4c'],
 'knownCards': ['ks', 'qd', 'ts', 'qs', '4c']}

In [216]:
firstTurn.revealBoard("js")


        Royal Flush:     2.174%
        Straight Flush:  2.174%
        Four of a Kind:  0.000%
        Full House:      0.000%
        Flush:           0.000%
        Straight:        0.000%
        Three of a Kind: 0.000%
        Two Pair:        0.000%
        Pair:            0.000%
        High Card:       [13, 12, 12, 11, 10, 4]
        


In [217]:
firstTurn.revealBoard("as")


        Royal Flush:     100.000%
        Straight Flush:  0.000%
        Four of a Kind:  0.000%
        Full House:      0.000%
        Flush:           0.000%
        Straight:        0.000%
        Three of a Kind: 0.000%
        Two Pair:        0.000%
        Pair:            0.000%
        High Card:       [14, 13, 12, 12, 11, 10, 4]
        
Best Result - Royal Flush
Your Hand: ['ks', 'qd']
Board: ['ts', 'qs', '4c', 'js', 'as']
With from board: 
